In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import nltk.sentiment
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

import os
import unicodedata
import re
import json

import j_acquire
import j_prep
import explore

In [2]:
data = j_acquire.scrape_github_data('repo_source')
df = pd.DataFrame.from_dict(data)
df = df.dropna()
df

,repo,language,readme_contents
1,tokyo-metropolitan-gov/covid19,Vue,# 東京都 新型コロナウイルス感染症対策サイト\n\n![](https://github....
3,covid19india/covid19india-react,JavaScript,"<p align=""center"">\n<img src=""https://lh3.goog..."
5,ieee8023/covid-chestxray-dataset,Jupyter Notebook,\n#### 🛑 Note: please do not claim diagnostic ...
7,ahmadawais/corona-cli,JavaScript,"<h4 align=""center"">\n <a href=""https://gith..."
8,ExpDev07/coronavirus-tracker-api,Python,"<h1 align=""center"">\n Coronavirus Tracker A..."
9,neherlab/covid19_scenarios,TypeScript,"<h1 id=""covid19_scenarios"" align=""center"">\n ..."
10,geohot/corona,Python,# Reverse engineering the coronavirus (SARS-Co...
11,soroushchehresa/awesome-coronavirus,JavaScript,"<div align=""center"">\n\t<br>\n\t<img src=""http..."
12,mhdhejazi/CoronaTracker,Swift,"<img src=""https://user-images.githubuserconten..."
13,pomber/covid19,JavaScript,Transforms the data from [CSSEGISandData/COVID...


In [3]:
df = j_prep.prep_readme_data(df)

In [4]:
df

,repo,language,readme_contents,basic_clean,clean_tokes,lemmatized,clean_lemmatized
1,tokyo-metropolitan-gov/covid19,Vue,# 東京都 新型コロナウイルス感染症対策サイト\n\n![](https://github....,\n\nhttpsgithubcomtokyometropolitangovcovid1...,[httpsgithubcomtokyometropolitangovcovid19work...,httpsgithubcomtokyometropolitangovcovid19workf...,httpsgithubcomtokyometropolitangovcovid19workf...
3,covid19india/covid19india-react,JavaScript,"<p align=""center"">\n<img src=""https://lh3.goog...",\n\n\n\n\n heres our data api\n \n\n setup\n\...,"[heres, our, data, api, setup, npm, i, npm, st...",here our data api setup npm i npm start mainta...,data api setup npm npm start maintainer jeremy...
5,ieee8023/covid-chestxray-dataset,Jupyter Notebook,\n#### 🛑 Note: please do not claim diagnostic ...,\n note please do not claim diagnostic perfor...,"[note, please, do, not, claim, diagnostic, per...",note please do not claim diagnostic performanc...,note please claim diagnostic performance model...
7,ahmadawais/corona-cli,JavaScript,"<h4 align=""center"">\n <a href=""https://gith...",\n \n \n \n \n \n\ntrack th...,"[track, the, coronavirus, disease, covid19, or...",track the coronavirus disease covid19 or the n...,track coronavirus disease covid19 novel corona...
8,ExpDev07/coronavirus-tracker-api,Python,"<h1 align=""center"">\n Coronavirus Tracker A...",\n coronavirus tracker api\n\n\nprovides up...,"[coronavirus, tracker, api, provides, uptodate...",coronavirus tracker api provides uptodate data...,coronavirus tracker api provides uptodate data...
9,neherlab/covid19_scenarios,TypeScript,"<h1 id=""covid19_scenarios"" align=""center"">\n ...",\n covid19 scenarios\n\n\n\nmodels of covid19...,"[covid19, scenarios, models, of, covid19, outb...",covid19 scenario model of covid19 outbreak tra...,covid19 scenario model covid19 outbreak trajec...
10,geohot/corona,Python,# Reverse engineering the coronavirus (SARS-Co...,reverse engineering the coronavirus sarscov2\...,"[reverse, engineering, the, coronavirus, sarsc...",reverse engineering the coronavirus sarscov2 s...,reverse engineering coronavirus sarscov2 start...
11,soroushchehresa/awesome-coronavirus,JavaScript,"<div align=""center"">\n\t<br>\n\t<img src=""http...",\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\tsponsors\n\t\n...,"[&#9;, &#9;, &#9;, &#9;, &#9;, &#9;, &#9;, &#9...",sponsor awesome coronavirus awesom...,sponsor awesome coronavirus awesomehttpsawesom...
12,mhdhejazi/CoronaTracker,Swift,"<img src=""https://user-images.githubuserconten...",\n\nioshttpsimgshieldsiobadgeios1020blue\nmaco...,"[ioshttpsimgshieldsiobadgeios1020blue, macosht...",ioshttpsimgshieldsiobadgeios1020blue macoshttp...,ioshttpsimgshieldsiobadgeios1020blue macoshttp...
13,pomber/covid19,JavaScript,Transforms the data from [CSSEGISandData/COVID...,transforms the data from cssegisanddatacovid19...,"[transforms, the, data, from, cssegisanddataco...",transforms the data from cssegisanddatacovid19...,transforms data cssegisanddatacovid19httpsgith...


In [5]:
# df.clean_lemmatized

In [6]:
# df.readme_contents.replace(r'/^[\s\d\d]/', '')

In [7]:
# df.clean_lemmatized.str.count(r'py\b')

In [8]:
# df.clean_lemmatized.str.count(r'js\b')

In [9]:
df.language.value_counts()

Python              7
JavaScript          6
Jupyter Notebook    5
TypeScript          2
PHP                 1
Swift               1
HTML                1
Vue                 1
R                   1
Ruby                1
Name: language, dtype: int64

In [10]:
# df['py_extension'] = df.readme_contents.str.count(r'https*')

In [11]:
# df['link_counts'] = df.readme_contents.str.count(r'https*')

In [12]:
# re.findall(r'\[.*?\]', df.readme_contents)

In [13]:
# for line in df.readme_contents:
#     re.findall(r'\[.*?\]')


In [14]:
# contents = df.readme_contents.to_string()

# # regex = '\[(.+)\]\(([^ ]+)( "(.+)")?\)' 

# len(contents)

In [15]:
df2 = explore.make_initial_df('repo_source')

In [16]:
df2 = explore.add_new_columns(df2)

In [17]:
df2.head()

,index,repo,language,readme_contents,basic_clean,clean_tokes,lemmatized,clean_lemmatized,gen_language,without_numbers,num_words,num_unique_words,link_counts,py_extensions,js_extensions,ipynb_extensions
0,1,tokyo-metropolitan-gov/covid19,Vue,# 東京都 新型コロナウイルス感染症対策サイト\n\n![](https://github....,\n\nhttpsgithubcomtokyometropolitangovcovid1...,[httpsgithubcomtokyometropolitangovcovid19work...,httpsgithubcomtokyometropolitangovcovid19workf...,httpsgithubcomtokyometropolitangovcovid19workf...,other,httpsgithubcomtokyometropolitangovcovid19workf...,19,19,3,0,0,0
1,3,covid19india/covid19india-react,JavaScript,"<p align=""center"">\n<img src=""https://lh3.goog...",\n\n\n\n\n heres our data api\n \n\n setup\n\...,"[heres, our, data, api, setup, npm, i, npm, st...",here our data api setup npm i npm start mainta...,data api setup npm npm start maintainer jeremy...,JavaScript,data api setup npm npm start maintainer jeremy...,50,47,7,0,0,0
2,5,ieee8023/covid-chestxray-dataset,Jupyter Notebook,\n#### 🛑 Note: please do not claim diagnostic ...,\n note please do not claim diagnostic perfor...,"[note, please, do, not, claim, diagnostic, per...",note please do not claim diagnostic performanc...,note please claim diagnostic performance model...,Jupyter Notebook,note please claim diagnostic performance model...,663,384,31,0,0,0
3,7,ahmadawais/corona-cli,JavaScript,"<h4 align=""center"">\n <a href=""https://gith...",\n \n \n \n \n \n\ntrack th...,"[track, the, coronavirus, disease, covid19, or...",track the coronavirus disease covid19 or the n...,track coronavirus disease covid19 novel corona...,JavaScript,track coronavirus disease covid19 novel corona...,823,413,70,0,2,0
4,8,ExpDev07/coronavirus-tracker-api,Python,"<h1 align=""center"">\n Coronavirus Tracker A...",\n coronavirus tracker api\n\n\nprovides up...,"[coronavirus, tracker, api, provides, uptodate...",coronavirus tracker api provides uptodate data...,coronavirus tracker api provides uptodate data...,Python,coronavirus tracker api provides uptodate data...,1032,480,144,2,1,0


In [18]:
vectorized_df = explore.make_vectorized_df(df2)

In [19]:
vectorized_df.head()

,00,002,01,02,04162020,05,05132020,05262020,06012020,0no,...,zinc,zingming,zoom,zum,num_words,num_unique_words,link_counts,py_extensions,js_extensions,ipynb_extensions
0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,19,19,3,0,0,0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,50,47,7,0,0,0
2,0.630423,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.022515,...,0.0,0.0,0.0,0.0,663,384,31,0,0,0
3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,823,413,70,0,2,0
4,0.000000,0.0,0.014731,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,1032,480,144,2,1,0
